In [6]:
import numpy as np
import pytket as tk
from pytket import passes as tkp
from pytket import circuit as tkc
from pytket.circuit.display import render_circuit_jupyter as print_circ
from pytket.qasm import circuit_from_qasm_str, circuit_to_qasm,circuit_from_qasm
import os


def cliff_t_rebase() -> tkp.BasePass:
    """Pass to convert single-qubit gates to the Clifford+T gateset.

    Concretely, single-qubit gates will be one of H, Z, S or T. Could also be
    updraded to handle X and V. Have a try!

    pyTKET won't do this for you automatically, because this is not a universal
    gateset (only approximately universal).

    For our purpose, if the decomposition is not exact, we raise an error
    """
    cx_replacement = tk.Circuit(2).CX(0, 1)
    def tk1_replacement(a, b, c, eps=1e-6):
        # make sure the phases are in the range [0, 4)
        a, b, c = a % 4, b % 4, c % 4
        ret = tk.Circuit(1)
        def add_phase(f: float):
            while f > eps:
                if f + eps > 1.:
                    ret.Z(0)
                    f -= 1
                elif f + eps > 0.5:
                    ret.S(0)
                    f -= 0.5
                elif f + eps > 0.25:
                    ret.T(0)
                    f -= 0.25
                else:
                    break
            return f
        rest_c = add_phase(c)
        ret.H(0)
        rest_b = add_phase(b)
        ret.H(0)
        rest_a = add_phase(a)

        if abs(rest_a) > eps or abs(rest_b) > eps or abs(rest_c) > eps:
            raise ValueError("Phases are not multiples of pi/4")
        return ret

    return tkp.RebaseCustom(
        {tk.OpType.CX, tk.OpType.H, tk.OpType.Tdg, tk.OpType.T},
        cx_replacement=cx_replacement,
        tk1_replacement=tk1_replacement
    )

def qasm_ordering(qasm_file, output_file):
    qasm_string = read_qasm_file(qasm_file) # Call function to read .qasm file and save as a string
    qc_from_qasm = circuit_from_qasm_str(qasm_string, maxwidth=1000) # Turn str into circuit
    cliff_opt = tkp.SequencePass([cliff_t_rebase()])
    cliff_opt.apply(qc_from_qasm)
    ordered_qasm_file = circuit_to_qasm(qc_from_qasm, output_file)

def read_qasm_file(file_path):
    """
    Reads the content of a QASM file into a string
    """
    with open(file_path, 'r') as file:
        qasm_str = file.read()
    return qasm_str


# Create a circuit with a CCX gate
qc = tk.Circuit(3)
qc.CCX(0, 1, 2)

box_decomp = tkp.SequencePass([tkp.DecomposeBoxes(), cliff_t_rebase()])
box_decomp.apply(qc)
print_circ(qc)


We see that we obtain the expected decomposition of the CCX gate, which is also minimal in number of CNOT gates.<br>
If we use CliffordSimp, we also obtain the same decomposition, but written in the clifford basis with Tdag = ZST


In [7]:
box_decomp = tkp.SequencePass([tkp.CliffordSimp(), cliff_t_rebase()])
box_decomp.apply(qc)
print_circ(qc)